In [11]:
# download weights from: https://github.com/larq/zoo/releases/download/binary_alexnet-v0.2.0/binary_alexnet_weights.h5

import qkeras as q
from qkeras import utils as u
import tensorflow as tf
from tensorflow import keras
import numpy as np
import larq as lq
from tqdm import tqdm
import json
import os 
import cv2

In [15]:
path_alexnet = "../../binary_alexnet_weights.h5"
# create random dataset
test_data = np.random.randint(low=0, high=254, size=(128, 1, 224, 224, 3))

# load data from ImageNET dataset
def loadImageNetData(path, image_num):
    file_list = os.listdir(path)
    file_list.sort()
    print(len(file_list))
    file_list = file_list[0:image_num]
    im = []
    for i in tqdm(file_list):
        img = cv2.imread(os.path.join(path, i))
        img = cv2.resize(img, (224, 224))
        im.append(img)
    im = np.asarray(im)
    im = np.expand_dims(im, axis=1)
    return im

In [16]:
def calculate_MSE(res_qkeras, res_larq):
    qres = np.asarray(res_qkeras)
    lres = np.asarray(res_larq)    
    qres = np.squeeze(qres)
    lres = np.squeeze(lres)
    mse = []
    for i in range(0,qres.shape[0]):
        real = lres[i,:]
        pred = qres[i,:]
        mse.append(((real - pred)**2).mean())
    return mse

def calculateAbsoluteError(res_qkeras, res_larq):
    error_counter = 0
    for i, j in zip(res_qkeras, res_larq):
        pred = np.argmax(np.asarray(i))
        real = np.argmax(np.asarray(j))
    if pred != real:
        error_counter += 1
    return error_counter

In [17]:
def qkeras_conv_block(qkeras_alexNet, features, kernel_size, pool, qnt, strides=1):
    if qnt:
        qkeras_alexNet.add(q.QActivation("binary"))
    qkeras_alexNet.add(q.QConv2D(features, kernel_size,strides=strides, padding="same", use_bias=False, kernel_quantizer="binary"))
    if pool:
        qkeras_alexNet.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
    qkeras_alexNet.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    return qkeras_alexNet

def qkeras_dense_block(qkeras_alexNet, units):
    qkeras_alexNet.add(q.QActivation("binary"))
    qkeras_alexNet.add(q.QDense(units, kernel_quantizer="binary", use_bias=False))
    qkeras_alexNet.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    return qkeras_alexNet

def larq_conv_block(larq_alexNet, features, kernel_size, pool, qnt, strides=1):
    larq_alexNet.add(lq.layers.QuantConv2D(features, kernel_size,strides=strides, padding="same", use_bias=False, 
                                            input_quantizer=None if not qnt else "ste_sign",
                                            kernel_quantizer="ste_sign",
                                            kernel_constraint="weight_clip"))
    if pool:
        larq_alexNet.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
    larq_alexNet.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    return larq_alexNet

def larq_dense_block(larq_alexNet, units):
    larq_alexNet.add(lq.layers.QuantDense(units, use_bias=False,
                                        input_quantizer="ste_sign",
                                        kernel_quantizer="ste_sign",
                                        kernel_constraint="weight_clip"))
    larq_alexNet.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    return larq_alexNet


In [18]:
qkeras_alexNet = tf.keras.models.Sequential() 
qkeras_alexNet.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))
qkeras_conv_block(qkeras_alexNet, features=64, kernel_size=11, strides=4, pool=True, qnt=False)
qkeras_conv_block(qkeras_alexNet, features=192, kernel_size=5, pool=True, qnt=True)
qkeras_conv_block(qkeras_alexNet, features=384, kernel_size=3, pool=False, qnt=True)
qkeras_conv_block(qkeras_alexNet, features=384, kernel_size=3, pool=False, qnt=True)
qkeras_conv_block(qkeras_alexNet, features=256, kernel_size=3, pool=True, qnt=True)

qkeras_alexNet.add(tf.keras.layers.Flatten())

qkeras_dense_block(qkeras_alexNet, units=4096)
qkeras_dense_block(qkeras_alexNet, units=4096)
qkeras_dense_block(qkeras_alexNet, units=1000)

qkeras_alexNet.add(tf.keras.layers.Activation("softmax", dtype="float32"))

In [19]:
larq_alexNet = tf.keras.models.Sequential() 
larq_alexNet.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))
larq_conv_block(larq_alexNet, features=64, kernel_size=11, strides=4, pool=True, qnt=False)
larq_conv_block(larq_alexNet, features=192, kernel_size=5, pool=True, qnt=True)
larq_conv_block(larq_alexNet, features=384, kernel_size=3, pool=False, qnt=True)
larq_conv_block(larq_alexNet, features=384, kernel_size=3, pool=False, qnt=True)
larq_conv_block(larq_alexNet, features=256, kernel_size=3, pool=True, qnt=True)

larq_alexNet.add(tf.keras.layers.Flatten())

larq_dense_block(larq_alexNet, units=4096)
larq_dense_block(larq_alexNet, units=4096)
larq_dense_block(larq_alexNet, units=1000)

larq_alexNet.add(tf.keras.layers.Activation("softmax", dtype="float32"))

In [20]:
qkeras_alexNet.load_weights(path_alexnet)
larq_alexNet.load_weights(path_alexnet)

# uncomment following line to test with imageNET data
#test_data = loadImageNetData("../../ILSVRC2012_img_val", 2000)

In [21]:
res_qkeras = []
res_larq = []

sample_num = 100

for i in tqdm(range(0,sample_num)):
    res_qkeras.append(qkeras_alexNet.predict(test_data[i,:,:,:]))
    res_larq.append(larq_alexNet.predict(test_data[i,:,:,:]))   
mse = calculate_MSE(res_qkeras, res_larq)
print("\n\nMSE for quicknet -> ", np.asarray(mse).mean())
print("Absolute errors for quicknet -> ", calculateAbsoluteError(res_qkeras, res_larq))

100%|██████████| 100/100 [01:30<00:00,  1.11it/s]



MSE for quicknet ->  0.00064042746
Absolute errors for quicknet ->  1


In [12]:
# save all models in topology (.txt), weights (.h5) format
with lq.context.quantized_scope(True):
    larq_alexNet.save("./larq_models/larq_alexNet.h5")  

larq_alexNet = larq_alexNet.to_json()


with open('./larq_models/larq_alexNet.txt', 'w') as outfile:
    json.dump(larq_alexNet, outfile)



u.model_save_quantized_weights(qkeras_alexNet, filename="./qkeras_models/qkeras_alexNet.h5")

qkeras_alexNet = qkeras_alexNet.to_json()

with open('./qkeras_models/qkeras_alexNet.txt', 'w') as outfile:
    json.dump(qkeras_alexNet, outfile)

AttributeError: 'str' object has no attribute 'save'